In [ ]:
import torch

from data_factory import *
from model_factory.models import *
from train import *
import matplotlib.pyplot as plt

In [ ]:
## Create training and validation datasets
# MMDataset creates a PyTorch Dataset object
# size is the width of each frame---either 32 or 64
# num_frames_input is the number of input frames---should always be set to 10
size = 32
num_frames_input = 10
batch_size = 32

train_dataset = MMDataset("data_factory/data/moving-mnist-train.npz", size = size)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size)

valid_dataset = MMDataset("data_factory/data/moving-mnist-valid.npz", size = size)
valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size = batch_size)

/usr/local/lib/python3.6/dist-packages/torchvision/transforms/functional.py:92: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  img = torch.from_numpy(np.array(pic, np.float32, copy=False))


In [ ]:
## Load the Baseline model
# Any other model can be loaded by changing the model name
# A full list of model names can be found in model_factory/models in the format "model_name.py"
model = Baseline().cuda()

## Choose training mode to be "reg" or "gt"
# For all non-autoregressive models, mode should be set to "reg"
# For autoregressive models:
#           -- "gt" takes ground truth frames as input at each step, and
#           -- "reg" takes the model's own predictions as input.
mode = "reg"

# Training hyperparameters
epochs = 40
lr = 1e-3
optimizer = torch.optim.Adam(lr = lr, params = model.parameters())
criterion = nn.MSELoss()

# Lists for storing training and validation history
loss_train_history = []
loss_valid_history = []


In [ ]:
# Train and cross-validate

for e in range(epochs):
  loss_train = train_once(model, criterion, num_frames_input, train_dataloader, optimizer, mode)
  loss_train_history.append(loss_train)

  loss_valid = valid_once(model, criterion, num_frames_input, valid_dataloader)
  loss_valid_history.append(loss_valid)